# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [21]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [22]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv(r"C:\Users\kylem\Desktop\Bootcamp_Assignments\Challenges\Challenge_6\python-api-challenge\weather_output_data\cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,edinburgh of the seven seas,-37.0676,-12.3116,10.42,67,43,18.85,SH,1691352199
1,1,tibanbang,6.6315,126.1055,27.47,79,100,3.52,PH,1691352200
2,2,bredasdorp,-34.5322,20.0403,10.56,93,0,1.26,ZA,1691352201
3,3,albany,42.6001,-73.9662,28.78,45,100,0.45,US,1691352080
4,4,pocitos,-34.9097,-56.1522,18.08,87,75,6.17,UY,1691352132


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [23]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color = "City"
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [24]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_temp_df = pd.DataFrame(city_data_df.loc[city_data_df['Max Temp'].between(21,27)])
ideal_wind_df = pd.DataFrame(ideal_temp_df.loc[ideal_temp_df['Wind Speed'] < 4.5])
ideal_df = pd.DataFrame(ideal_wind_df.loc[ideal_wind_df['Cloudiness'] == 0])

# Drop any rows with null values
ideal_df = ideal_df.dropna()

# Display sample data
ideal_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
21,21,taquari,-29.7997,-51.8644,22.95,64,0,1.20,BR,1691352210
35,35,bingol,38.8847,40.4939,25.11,23,0,0.56,TR,1691352216
116,116,mil'kovo,55.6078,37.7956,25.11,70,0,3.55,RU,1691352251
147,147,pisco,-13.7000,-76.2167,21.03,68,0,4.12,PE,1691352266
178,178,watford city,47.8022,-103.2832,25.09,43,0,3.09,US,1691352021


### Step 3: Create a new DataFrame called `hotel_df`.

In [25]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
21,taquari,BR,-29.7997,-51.8644,64,
35,bingol,TR,38.8847,40.4939,23,
116,mil'kovo,RU,55.6078,37.7956,70,
147,pisco,PE,-13.7000,-76.2167,68,
178,watford city,US,47.8022,-103.2832,43,
183,lucapa,AO,-8.4192,20.7447,30,
193,argo,GR,37.6333,22.7333,56,
204,tazacorte,ES,28.6290,-17.9293,83,
213,campos novos,BR,-27.4017,-51.2250,58,
261,talas,TR,38.6908,35.5538,57,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [27]:
# Set parameters to search for a hotel
radius = 10000
limit = 1
params = {
    "limit" : limit,
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = hotel_df.loc[index,'Lat']
    lon = hotel_df.loc[index,'Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lon},{lat},{radius}'
    params["bias"] = f'proximity:{lon},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
taquari - nearest hotel: No hotel found
bingol - nearest hotel: Grand Hotel Berti
mil'kovo - nearest hotel: Дрозды
pisco - nearest hotel: La Portada
watford city - nearest hotel: No hotel found
lucapa - nearest hotel: No hotel found
argo - nearest hotel: Apollon Hotel
tazacorte - nearest hotel: App Leyma
campos novos - nearest hotel: No hotel found
talas - nearest hotel: PRIME INN
constantia - nearest hotel: Hotel Chérica
albertina - nearest hotel: Hotel Parque das Primaveras
yuci - nearest hotel: 7 Days Inn Jinzhong
gulnar - nearest hotel: No hotel found
sivrice - nearest hotel: No hotel found
serhetabat - nearest hotel: No hotel found
l'alcora - nearest hotel: No hotel found
lata - nearest hotel: Residencial Botânico


,City,Country,Lat,Lng,Humidity,Hotel Name
21,taquari,BR,-29.7997,-51.8644,64,No hotel found
35,bingol,TR,38.8847,40.4939,23,Grand Hotel Berti
116,mil'kovo,RU,55.6078,37.7956,70,Дрозды
147,pisco,PE,-13.7000,-76.2167,68,La Portada
178,watford city,US,47.8022,-103.2832,43,No hotel found
183,lucapa,AO,-8.4192,20.7447,30,No hotel found
193,argo,GR,37.6333,22.7333,56,Apollon Hotel
204,tazacorte,ES,28.6290,-17.9293,83,App Leyma
213,campos novos,BR,-27.4017,-51.2250,58,No hotel found
261,talas,TR,38.6908,35.5538,57,PRIME INN


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [36]:
%%capture --no-display

# Configure the map plot
hotel_plot = hotel_df.hvplot.points('Lng',
    'Lat', 
    geo=True, 
    tiles = 'OSM', 
    frame_width = 700,
    frame_height = 700,
    size = 'Humidity',
    color = 'City',
    hover_cols = ['Hotel Name', 'Country'])

hotel_plot

# Display the map
hotel_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)